In [85]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from geopy.geocoders import Nominatim
from geopy import geocoders
import geopandas
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon
import geocoder
import re
import usaddress
import datetime
import calendar
from shapely.geometry import Point, Polygon

Combine tract map with census data

In [86]:
census_tract_map_file = r'2010 Census Tracts\geo_export_45d3700d-1562-4a98-b26f-0c47aaf39979.shp'
tract_map = geopandas.GeoDataFrame.from_file(census_tract_map_file)

In [87]:
tract_map.head()

,boro_code,boro_ct201,boro_name,cdeligibil,ct2010,ctlabel,ntacode,ntaname,puma,shape_area,shape_leng,geometry
0,5,5000900,Staten Island,E,000900,9,SI22,West New Brighton-New Brighton-St. George,3903,2.497010e+06,7729.016794,"POLYGON ((-74.07921 40.64343, -74.07914 40.643..."
1,1,1009800,Manhattan,I,009800,98,MN19,Turtle Bay-East Midtown,3808,1.906016e+06,5534.199811,"POLYGON ((-73.96433 40.75638, -73.96479 40.755..."
2,1,1010000,Manhattan,I,010000,100,MN19,Turtle Bay-East Midtown,3808,1.860938e+06,5692.168737,"POLYGON ((-73.96802 40.75958, -73.96848 40.758..."
3,1,1010200,Manhattan,I,010200,102,MN17,Midtown-Midtown South,3807,1.860993e+06,5687.802439,"POLYGON ((-73.97124 40.76094, -73.97170 40.760..."
4,1,1010400,Manhattan,I,010400,104,MN17,Midtown-Midtown South,3807,1.864600e+06,5693.036367,"POLYGON ((-73.97446 40.76229, -73.97491 40.761..."


In [88]:
tract_map['boro_name'].unique()

array(['Staten Island', 'Manhattan', 'Brooklyn', 'Bronx', 'Queens'],
      dtype=object)

In [89]:
county_dic = {'Bronx':'Bronx','Brooklyn':'Kings','Manhattan':'New York','Staten Island':'Richmond','Queens':'Queens'}
tract_map['county_name']=tract_map['boro_name'].replace(county_dic)

In [90]:
#it is not a problem that the id value is treated as a string, its only used to generate a unique id
tract_map['ctlabel']

0             9
1            98
2           100
3           102
4           104
         ...   
2160        212
2161    1072.01
2162    1072.02
2163        226
2164        248
Name: ctlabel, Length: 2165, dtype: object

In [91]:
tract_map['tract_id']="Census Tract " +tract_map['ctlabel']+', '+tract_map['county_name']+' County, New York'

In [92]:
census_data = pd.read_csv('cleaned_census_data.csv')

census_data=census_data.set_index('Geographic Area Name')

census_data.head()

,id,Total population,Median age (years),Sex ratio (males per 100 females),Age dependency ratio,Occupied housing units,Owner-occupied housing units,Renter-occupied housing units,Households below poverty level (12 months),Median household income (2018 inflation adjusted),...,Some other race alone,Hispanic or Latino,white_pop_%,black_pop_%,native_american_pop_%,asian_pop_%,other_race_%,hispanic_pop_%,%_owner_housing,%_rental_housing
Geographic Area Name,,,,,,,,,,,,,,,,,,,,,
"Census Tract 1, Albany County, New York",1400000US36001000100,2022,33.6,86.0,71.2,837,216,621,253,29063.0,...,21,412,0.233927,0.577151,0.043521,0.026706,0.010386,0.203759,0.258065,0.741935
"Census Tract 2, Albany County, New York",1400000US36001000200,4700,30.7,75.5,42.0,2133,408,1725,688,29470.0,...,0,268,0.132128,0.762340,0.000000,0.015532,0.000000,0.057021,0.191280,0.808720
"Census Tract 3, Albany County, New York",1400000US36001000300,5966,37.2,88.5,64.6,2480,914,1566,572,37296.0,...,238,628,0.423232,0.322829,0.002849,0.104760,0.039893,0.105263,0.368548,0.631452
"Census Tract 4.01, Albany County, New York",1400000US36001000401,2479,65.2,58.3,137.9,1029,673,356,60,75809.0,...,8,127,0.860428,0.093990,0.006051,0.027430,0.003227,0.051230,0.654033,0.345967
"Census Tract 4.03, Albany County, New York",1400000US36001000403,4236,37.4,101.1,39.1,2172,1144,1028,306,70488.0,...,104,366,0.733711,0.114259,0.005666,0.047214,0.024551,0.086402,0.526703,0.473297


In [93]:
tract_map_merged =tract_map.join(other=census_data,on='tract_id',how='left',rsuffix='_2')

In [94]:
tract_map_merged.columns

Index(['boro_code', 'boro_ct201', 'boro_name', 'cdeligibil', 'ct2010',
       'ctlabel', 'ntacode', 'ntaname', 'puma', 'shape_area', 'shape_leng',
       'geometry', 'county_name', 'tract_id', 'id', 'Total population',
       'Median age (years)', 'Sex ratio (males per 100 females)',
       'Age dependency ratio', 'Occupied housing units',
       'Owner-occupied housing units', 'Renter-occupied housing units',
       'Households below poverty level (12 months)',
       'Median household income (2018 inflation adjusted)',
       'Households receiving food stamps (12 months)', 'Families',
       'White alone', 'Black or African American alone',
       'American Indian and Alaska Native alone', 'Asian alone',
       'Some other race alone', 'Hispanic or Latino', 'white_pop_%',
       'black_pop_%', 'native_american_pop_%', 'asian_pop_%', 'other_race_%',
       'hispanic_pop_%', '%_owner_housing', '%_rental_housing'],
      dtype='object')

Group Evictions by census tract id

In [95]:
eviction_data = pd.read_csv('evictions_geocoded_2.csv',index_col='Unnamed: 0')

C:\Users\Public\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (13,16) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [96]:
eviction_data.head()

,COURT_INDEX_NUMBER,DOCKET_NUMBER,EVICTION_ADDRESS,EVICTION_APT_NUM,EXECUTED_DATE,MARSHAL_FIRST_NAME,MARSHAL_LAST_NAME,RESIDENTIAL_COMMERCIAL_IND,BOROUGH,EVICTION_ZIP,...,eviction_address2,full_address,lat_lon,lat_lon2,lat_lon_both,lat,lon,Executed_Year,Executed_Month,executed_year_month
0,50180/17,5469,463 LEXINGTON AVENUE APT. 1,1,2017-09-12,Robert,Renzulli,Residential,BROOKLYN,11221,...,NaN,"463 LEXINGTON AVENUE, BROOKLYN, NY, 11221","(40.68865999611498, -73.94358600207771)",NaN,"(40.68865999611498, -73.94358600207771)",40.68866,-73.943586,2017,9,2017-09-01
1,73750/18B,93847,448 SAPPHIRE STREET,1,2019-06-03,Justin,Grossman,Residential,BROOKLYN,11208,...,NaN,"448 SAPPHIRE STREET, BROOKLYN, NY, 11208","(40.66745998368947, -73.8567509939227)",NaN,"(40.66745998368947, -73.8567509939227)",40.66746,-73.856751,2019,6,2019-06-01
2,K65227/17,97709,1036 PRESIDENT ST,B2B,2017-11-16,Darlene,Barone,Residential,BROOKLYN,11225,...,NaN,"1036 PRESIDENT ST, BROOKLYN, NY, 11225","(40.668459986647804, -73.95718496942762)",NaN,"(40.668459986647804, -73.95718496942762)",40.66846,-73.957185,2017,11,2017-11-01
3,51881/18,350940,2124 ST. PAUL AVENUE,2ND FLOOR,2019-02-05,Thomas,Bia,Residential,BRONX,10461,...,NaN,"2124 ST. PAUL AVENUE, BRONX, NY, 10461","(40.85483998565695, -73.82997902754228)",NaN,"(40.85483998565695, -73.82997902754228)",40.85484,-73.829979,2019,2,2019-02-01
4,66981/18,351548,1098 GRANT AVENUE,2,2019-06-07,Thomas,Bia,Residential,BRONX,10456,...,NaN,"1098 GRANT AVENUE, BRONX, NY, 10456","(40.83090001290239, -73.91641002796423)",NaN,"(40.83090001290239, -73.91641002796423)",40.83090,-73.916410,2019,6,2019-06-01


In [97]:
geo_eviction_data = geopandas.GeoDataFrame(
    eviction_data, geometry=geopandas.points_from_xy(eviction_data['lon'],eviction_data['lat']))

In [98]:
geo_eviction_data=geo_eviction_data.dropna(subset=['lat_lon_both'])

In [99]:
geo_eviction_data.crs = tract_map_merged.crs

In [100]:
eviction_geo_tracts = geopandas.sjoin(geo_eviction_data, tract_map_merged, how="left", op='within',rsuffix='_tract')

C:\Users\Public\Anaconda\lib\site-packages\numpy\lib\function_base.py:2167: RuntimeWarning: invalid value encountered in ? (vectorized)
  outputs = ufunc(*inputs)


In [101]:
eviction_geo_tracts.head()

,COURT_INDEX_NUMBER,DOCKET_NUMBER,EVICTION_ADDRESS,EVICTION_APT_NUM,EXECUTED_DATE,MARSHAL_FIRST_NAME,MARSHAL_LAST_NAME,RESIDENTIAL_COMMERCIAL_IND,BOROUGH,EVICTION_ZIP,...,Some other race alone,Hispanic or Latino,white_pop_%,black_pop_%,native_american_pop_%,asian_pop_%,other_race_%,hispanic_pop_%,%_owner_housing,%_rental_housing
0,50180/17,5469,463 LEXINGTON AVENUE APT. 1,1,2017-09-12,Robert,Renzulli,Residential,BROOKLYN,11221,...,117.0,253.0,0.260846,0.622017,0.017354,0.022234,0.063449,0.137202,0.161520,0.838480
1,73750/18B,93847,448 SAPPHIRE STREET,1,2019-06-03,Justin,Grossman,Residential,BROOKLYN,11208,...,198.0,732.0,0.054413,0.894813,0.000000,0.008189,0.036033,0.133212,0.163671,0.836329
2,K65227/17,97709,1036 PRESIDENT ST,B2B,2017-11-16,Darlene,Barone,Residential,BROOKLYN,11225,...,158.0,449.0,0.076440,0.852206,0.000000,0.026627,0.023635,0.067165,0.084660,0.915340
3,51881/18,350940,2124 ST. PAUL AVENUE,2ND FLOOR,2019-02-05,Thomas,Bia,Residential,BRONX,10461,...,2223.0,3527.0,0.447110,0.100450,0.002832,0.058971,0.370315,0.587540,0.265565,0.734435
4,66981/18,351548,1098 GRANT AVENUE,2,2019-06-07,Thomas,Bia,Residential,BRONX,10456,...,1351.0,1890.0,0.050531,0.476022,0.027358,0.000000,0.434825,0.608304,0.074641,0.925359


In [102]:
eviction_geo_tracts.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 66338 entries, 0 to 66348
Data columns (total 63 columns):
 #   Column                                             Non-Null Count  Dtype   
---  ------                                             --------------  -----   
 0   COURT_INDEX_NUMBER                                 66338 non-null  object  
 1   DOCKET_NUMBER                                      66338 non-null  int64   
 2   EVICTION_ADDRESS                                   66338 non-null  object  
 3   EVICTION_APT_NUM                                   55373 non-null  object  
 4   EXECUTED_DATE                                      66338 non-null  object  
 5   MARSHAL_FIRST_NAME                                 66338 non-null  object  
 6   MARSHAL_LAST_NAME                                  66338 non-null  object  
 7   RESIDENTIAL_COMMERCIAL_IND                         66338 non-null  object  
 8   BOROUGH                                            66338 non-null  o

In [103]:
eviction_geo_tracts=eviction_geo_tracts.drop(columns=['lat_lon', 'lat_lon2','lat_lon_both','eviction_address2','id'])

In [104]:
eviction_geo_tracts.to_file("eviction_geo_df.geojson", driver='GeoJSON')

In [105]:
#Census tracts do not fall within a single zipcode
eviction_geo_tracts[['EVICTION_ZIP','tract_id']].groupby('tract_id').mean()

,EVICTION_ZIP
tract_id,
"Census Tract 1, Kings County, New York",11201.000000
"Census Tract 1, Queens County, New York",11106.714286
"Census Tract 10, Queens County, New York",11421.000000
"Census Tract 10.02, New York County, New York",10002.280000
"Census Tract 100, Kings County, New York",11218.785714
...,...
"Census Tract 997.04, Queens County, New York",11360.000000
"Census Tract 997.05, Queens County, New York",11360.000000
"Census Tract 998, Kings County, New York",11236.000000


join evictions groupby dataframe to census tract dataframe

In [106]:
sub_eviction_geo_tracts=eviction_geo_tracts[['COURT_INDEX_NUMBER','tract_id']].groupby('tract_id').count()
sub_eviction_geo_tracts=sub_eviction_geo_tracts.rename(columns={'COURT_INDEX_NUMBER':'#_evictions'})

In [107]:
sub_eviction_geo_tracts.head()

,#_evictions
tract_id,
"Census Tract 1, Kings County, New York",4
"Census Tract 1, Queens County, New York",49
"Census Tract 10, Queens County, New York",20
"Census Tract 10.02, New York County, New York",25
"Census Tract 100, Kings County, New York",14


In [108]:
eviction_census_df=tract_map_merged.join(other=sub_eviction_geo_tracts,on='tract_id')
eviction_census_df['#_evictions']=eviction_census_df['#_evictions'].fillna(0)

In [109]:
eviction_census_df.head()

,boro_code,boro_ct201,boro_name,cdeligibil,ct2010,ctlabel,ntacode,ntaname,puma,shape_area,...,Hispanic or Latino,white_pop_%,black_pop_%,native_american_pop_%,asian_pop_%,other_race_%,hispanic_pop_%,%_owner_housing,%_rental_housing,#_evictions
0,5,5000900,Staten Island,E,000900,9,SI22,West New Brighton-New Brighton-St. George,3903,2.497010e+06,...,445,0.339540,0.471522,0.000000,0.122673,0.003834,0.243702,0.405882,0.594118,40.0
1,1,1009800,Manhattan,I,009800,98,MN19,Turtle Bay-East Midtown,3808,1.906016e+06,...,424,0.830972,0.020000,0.000972,0.127222,0.010139,0.058889,0.448366,0.551634,39.0
2,1,1010000,Manhattan,I,010000,100,MN19,Turtle Bay-East Midtown,3808,1.860938e+06,...,61,0.792421,0.019796,0.000000,0.171946,0.000000,0.034502,0.234835,0.765165,17.0
3,1,1010200,Manhattan,I,010200,102,MN17,Midtown-Midtown South,3807,1.860993e+06,...,11,0.640000,0.000000,0.230000,0.020000,0.000000,0.110000,0.551020,0.448980,17.0
4,1,1010400,Manhattan,I,010400,104,MN17,Midtown-Midtown South,3807,1.864600e+06,...,103,0.704026,0.023939,0.000000,0.194777,0.062024,0.112078,0.499124,0.500876,14.0


In [110]:
eviction_census_df.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 2165 entries, 0 to 2164
Data columns (total 41 columns):
 #   Column                                             Non-Null Count  Dtype   
---  ------                                             --------------  -----   
 0   boro_code                                          2165 non-null   object  
 1   boro_ct201                                         2165 non-null   object  
 2   boro_name                                          2165 non-null   object  
 3   cdeligibil                                         2164 non-null   object  
 4   ct2010                                             2165 non-null   object  
 5   ctlabel                                            2165 non-null   object  
 6   ntacode                                            2165 non-null   object  
 7   ntaname                                            2165 non-null   object  
 8   puma                                               2165 non-null   obj

In [111]:
eviction_census_df.to_file("eviction_census_no_dates.geojson", driver='GeoJSON')

In [112]:
eviction_census_df.to_csv('eviction_census_no_dates.csv',index=False)

Generate monthly dataset

In [113]:
eviction_data['executed_year_month']=pd.to_datetime(eviction_data['executed_year_month'])

In [114]:
eviction_data['executed_year_month'].min()

Timestamp('2017-01-01 00:00:00')

In [115]:
eviction_data['executed_year_month'].max()

Timestamp('2020-12-01 00:00:00')

In [116]:
year_list = eviction_data['executed_year_month'].dt.year.unique()
year_list.sort()
year_list

array([2017, 2018, 2019, 2020], dtype=int64)

In [117]:
month_list =eviction_data['executed_year_month'].dt.month.unique()
month_list.sort()
month_list

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12], dtype=int64)

In [118]:
year_month_list = []
for year in year_list:
    for month in month_list:
        year_month_list.append(str(year)+'_'+str(month))

In [119]:
year_month_list[0:5]

['2017_1', '2017_2', '2017_3', '2017_4', '2017_5']

In [120]:
census_id_list=tract_map_merged['tract_id'].unique()

In [121]:
census_tract_year_month_list = []
for tract in census_id_list:
    for year_month in year_month_list:
        census_tract_year_month_list.append(tract+'__'+year_month)

In [122]:
census_tract_date_df=pd.DataFrame(data=census_tract_year_month_list,columns=['census_tract_date_id'])

In [123]:
census_tract_date_df['census_tract']=census_tract_date_df['census_tract_date_id'].str.split('__',expand=True)[0]

In [124]:
#adding static census values
census_tract_date_df=census_tract_date_df.join(other=tract_map_merged.set_index('tract_id'),on='census_tract')

In [125]:
census_tract_date_df.columns

Index(['census_tract_date_id', 'census_tract', 'boro_code', 'boro_ct201',
       'boro_name', 'cdeligibil', 'ct2010', 'ctlabel', 'ntacode', 'ntaname',
       'puma', 'shape_area', 'shape_leng', 'geometry', 'county_name', 'id',
       'Total population', 'Median age (years)',
       'Sex ratio (males per 100 females)', 'Age dependency ratio',
       'Occupied housing units', 'Owner-occupied housing units',
       'Renter-occupied housing units',
       'Households below poverty level (12 months)',
       'Median household income (2018 inflation adjusted)',
       'Households receiving food stamps (12 months)', 'Families',
       'White alone', 'Black or African American alone',
       'American Indian and Alaska Native alone', 'Asian alone',
       'Some other race alone', 'Hispanic or Latino', 'white_pop_%',
       'black_pop_%', 'native_american_pop_%', 'asian_pop_%', 'other_race_%',
       'hispanic_pop_%', '%_owner_housing', '%_rental_housing'],
      dtype='object')

In [126]:
census_tract_date_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103920 entries, 0 to 103919
Data columns (total 41 columns):
 #   Column                                             Non-Null Count   Dtype   
---  ------                                             --------------   -----   
 0   census_tract_date_id                               103920 non-null  object  
 1   census_tract                                       103920 non-null  object  
 2   boro_code                                          103920 non-null  object  
 3   boro_ct201                                         103920 non-null  object  
 4   boro_name                                          103920 non-null  object  
 5   cdeligibil                                         103872 non-null  object  
 6   ct2010                                             103920 non-null  object  
 7   ctlabel                                            103920 non-null  object  
 8   ntacode                                            103920 non-nu

In [127]:
eviction_geo_tracts['tract_date']=eviction_geo_tracts['tract_id'].astype(str)+'__'+eviction_geo_tracts['Executed_Year'].astype(str)+'_'+eviction_geo_tracts['Executed_Month'].astype(str)

In [128]:
eviction_geo_tracts['tract_date'][0:10]

0     Census Tract 263, Kings County, New York__2017_9
1    Census Tract 1220, Kings County, New York__2019_6
2    Census Tract 325, Kings County, New York__2017_11
3     Census Tract 300, Bronx County, New York__2019_2
4    Census Tract 181.01, Bronx County, New York__2...
5    Census Tract 11, Richmond County, New York__20...
6      Census Tract 37, Bronx County, New York__2017_1
7    Census Tract 138, New York County, New York__2...
8     Census Tract 220, Bronx County, New York__2017_8
9    Census Tract 433, Kings County, New York__2018_11
Name: tract_date, dtype: object

In [129]:
sub_data = pd.DataFrame(data=eviction_geo_tracts['tract_date'].value_counts())

In [130]:
sub_data=sub_data.rename(columns={'tract_date':'#_evictions'})

In [131]:
sub_data.head()

,#_evictions
"Census Tract 336, Bronx County, New York__2018_4",20
"Census Tract 185, Bronx County, New York__2017_1",17
"Census Tract 431, Bronx County, New York__2017_5",15
"Census Tract 1208, Kings County, New York__2017_1",14
"Census Tract 393, Bronx County, New York__2018_1",14


In [132]:
census_tract_date_df=census_tract_date_df.join(other=sub_data,on='census_tract_date_id')

In [133]:
census_tract_date_df.head()

,census_tract_date_id,census_tract,boro_code,boro_ct201,boro_name,cdeligibil,ct2010,ctlabel,ntacode,ntaname,...,Hispanic or Latino,white_pop_%,black_pop_%,native_american_pop_%,asian_pop_%,other_race_%,hispanic_pop_%,%_owner_housing,%_rental_housing,#_evictions
0,"Census Tract 9, Richmond County, New York__2017_1","Census Tract 9, Richmond County, New York",5,5000900,Staten Island,E,000900,9,SI22,West New Brighton-New Brighton-St. George,...,445,0.33954,0.471522,0.0,0.122673,0.003834,0.243702,0.405882,0.594118,2.0
1,"Census Tract 9, Richmond County, New York__2017_2","Census Tract 9, Richmond County, New York",5,5000900,Staten Island,E,000900,9,SI22,West New Brighton-New Brighton-St. George,...,445,0.33954,0.471522,0.0,0.122673,0.003834,0.243702,0.405882,0.594118,NaN
2,"Census Tract 9, Richmond County, New York__2017_3","Census Tract 9, Richmond County, New York",5,5000900,Staten Island,E,000900,9,SI22,West New Brighton-New Brighton-St. George,...,445,0.33954,0.471522,0.0,0.122673,0.003834,0.243702,0.405882,0.594118,2.0
3,"Census Tract 9, Richmond County, New York__2017_4","Census Tract 9, Richmond County, New York",5,5000900,Staten Island,E,000900,9,SI22,West New Brighton-New Brighton-St. George,...,445,0.33954,0.471522,0.0,0.122673,0.003834,0.243702,0.405882,0.594118,1.0
4,"Census Tract 9, Richmond County, New York__2017_5","Census Tract 9, Richmond County, New York",5,5000900,Staten Island,E,000900,9,SI22,West New Brighton-New Brighton-St. George,...,445,0.33954,0.471522,0.0,0.122673,0.003834,0.243702,0.405882,0.594118,2.0


In [134]:
census_tract_date_df.isna().mean()

census_tract_date_id                                 0.000000
census_tract                                         0.000000
boro_code                                            0.000000
boro_ct201                                           0.000000
boro_name                                            0.000000
cdeligibil                                           0.000462
ct2010                                               0.000000
ctlabel                                              0.000000
ntacode                                              0.000000
ntaname                                              0.000000
puma                                                 0.000000
shape_area                                           0.000000
shape_leng                                           0.000000
geometry                                             0.000000
county_name                                          0.000000
id                                                   0.000000
Total po

In [135]:
#if there is no record of an eviction in a tract_month, there are no evictions
census_tract_date_df['#_evictions']=census_tract_date_df['#_evictions'].fillna(0)

In [136]:
census_tract_date_df['year']=census_tract_date_df['census_tract_date_id'].str.split('_',expand=True)[2].astype(int)
census_tract_date_df['month']=census_tract_date_df['census_tract_date_id'].str.split('_',expand=True)[3].astype(int)

In [137]:
census_tract_date_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103920 entries, 0 to 103919
Data columns (total 44 columns):
 #   Column                                             Non-Null Count   Dtype   
---  ------                                             --------------   -----   
 0   census_tract_date_id                               103920 non-null  object  
 1   census_tract                                       103920 non-null  object  
 2   boro_code                                          103920 non-null  object  
 3   boro_ct201                                         103920 non-null  object  
 4   boro_name                                          103920 non-null  object  
 5   cdeligibil                                         103872 non-null  object  
 6   ct2010                                             103920 non-null  object  
 7   ctlabel                                            103920 non-null  object  
 8   ntacode                                            103920 non-nu

In [138]:
geo_eviction_data = geopandas.GeoDataFrame(
    census_tract_date_df, geometry=census_tract_date_df['geometry'])

In [139]:
geo_eviction_data.to_file("census_date_data.geojson", driver='GeoJSON')

In [140]:
census_tract_date_df['evictions_per_capita']=census_tract_date_df['#_evictions']/census_tract_date_df['Total population']
census_tract_date_df['evictions_per_housing_unit']=census_tract_date_df['#_evictions']/census_tract_date_df['Occupied housing units']
census_tract_date_df['evictions_per_family']=census_tract_date_df['#_evictions']/census_tract_date_df['Families']
census_tract_date_df['year_month']=census_tract_date_df['year'].astype(str)+'-'+census_tract_date_df['month'].astype(str)
census_tract_date_df['year_month']=pd.to_datetime(census_tract_date_df['year_month'])

In [141]:
census_tract_date_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103920 entries, 0 to 103919
Data columns (total 48 columns):
 #   Column                                             Non-Null Count   Dtype         
---  ------                                             --------------   -----         
 0   census_tract_date_id                               103920 non-null  object        
 1   census_tract                                       103920 non-null  object        
 2   boro_code                                          103920 non-null  object        
 3   boro_ct201                                         103920 non-null  object        
 4   boro_name                                          103920 non-null  object        
 5   cdeligibil                                         103872 non-null  object        
 6   ct2010                                             103920 non-null  object        
 7   ctlabel                                            103920 non-null  object        
 8   ntac

In [142]:
census_tract_date_df.to_csv('census_date_data.csv',index=False)